# Train Test Split

File name is represented as "index of the word" + "performer id" + "repetition"

In [ ]:
# Importing Libraries
import os
import shutil
from collections import defaultdict
import random
from tqdm import tqdm
import cv2
import numpy as np
import os
from tqdm import tqdm

In [ ]:
def split_videos_by_category(video_folder, train_folder, test_folder, split_ratio=0.8):
    # Creating training and testing folders
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # Grouping videos
    category_videos = defaultdict(list)
    for video_file in os.listdir(video_folder):
        if video_file.endswith('.mp4'):
            category = extract_category(video_file)
            category_videos[category].append(video_file)

    # Videos split - train test
    for category, videos in category_videos.items():
        num_videos = len(videos)
        num_train = int(num_videos * split_ratio)
        random.shuffle(videos)

        # Videos split - train test
        train_videos = videos[:num_train]
        test_videos = videos[num_train:]

        # Moving files to respective folders
        print(f"Processing category: {category}")
        progress_bar_train = tqdm(total=len(train_videos), desc="Copying training videos")
        for video in train_videos:
            source_path = os.path.join(video_folder, video)
            dest_path = os.path.join(train_folder, video)
            shutil.copy(source_path, dest_path)
            progress_bar_train.update(1)
        progress_bar_train.close()
        progress_bar_test = tqdm(total=len(test_videos), desc="Copying testing videos")
        for video in test_videos:
            source_path = os.path.join(video_folder, video)
            dest_path = os.path.join(test_folder, video)
            shutil.copy(source_path, dest_path)
            progress_bar_test.update(1)
        progress_bar_test.close()

def extract_category(video_file):
    category = video_file.split('_')[0]
    return category

if __name__ == "__main__":
    video_folder = os.getcwd() + "\\gesture_frame"
    train_folder = os.getcwd() + "\\train"
    test_folder = os.getcwd() + "\\test"

    # Split videos into training and testing data by category
    split_videos_by_category(video_folder, train_folder, test_folder, split_ratio=0.8)

Processing category: 001


Copying testing videos: 100%|██████████| 4/4 [00:00<00:00, 55.58it/s]


Processing category: 002


Copying testing videos: 100%|██████████| 3/3 [00:00<00:00, 82.92it/s]


# Video to Frame - Training Data

In [1]:
# Defining the boundaries for hand segmentation
boundaries = [
    ([0, 120, 0], [140, 255, 100]),
    ([25, 0, 75], [180, 38, 255])
]

def handsegment(frame):
    lower, upper = boundaries[0]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask1 = cv2.inRange(frame, lower, upper)

    lower, upper = boundaries[1]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")
    mask2 = cv2.inRange(frame, lower, upper)

    mask = cv2.bitwise_or(mask1, mask2)
    output = cv2.bitwise_and(frame, frame, mask=mask)

    return output

def convert_video_to_frames(video_path, output_folder, gesture_name, hc, pbar):
    # Opening the video file
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    if not success:
        print("Error: Failed to open video file.")
        return

    # Creating a subfolder for each video's frames
    video_frame_folder = os.path.join(output_folder, os.path.splitext(os.path.basename(video_path))[0])
    os.makedirs(video_frame_folder, exist_ok=True)

    # Getting total number of frames in the video
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    while success:
        # saving the grayscale frames
        frame_path = os.path.join(video_frame_folder, f"frame{count}.jpg")
        cv2.imwrite(frame_path, image)

        # Append information to list
        hc.append({
            "file_path": frame_path,
            "gesture": gesture_name,
            "total_frame_count": count
        })
        # conducting hand segment
        segmented_frame = handsegment(image)
        gray_frame = cv2.cvtColor(segmented_frame, cv2.COLOR_BGR2GRAY)

        # Saving the segmented frame
        cv2.imwrite(frame_path, gray_frame)
        pbar.update(1)  # Update progress bar
        success, image = vidcap.read()
        count += 1

    # Releasing video capture and destroy all OpenCV windows
    vidcap.release()
    cv2.destroyAllWindows()

def extract_category(video_file):
    category = video_file.split('_')[0]
    return category

output_folder = os.path.join(train_folder, "frames_output")
os.makedirs(output_folder, exist_ok=True)

hc = []

pbar_total = len([filename for filename in os.listdir(train_folder) if filename.endswith(".mp4")])
pbar = tqdm(total=pbar_total, desc="Converting frames")

for filename in os.listdir(train_folder):
    if filename.endswith(".mp4"):
        category = extract_category(filename)
        category_output_folder = os.path.join(output_folder, category)
        os.makedirs(category_output_folder, exist_ok=True)
        video_path = os.path.join(train_folder, filename)
        convert_video_to_frames(video_path, category_output_folder, category, hc, pbar)

# Video to Frame - Test data

In [ ]:
output_folder = os.path.join(test_folder, "frames_output")
os.makedirs(output_folder, exist_ok=True)
hc = []

pbar_total = len([filename for filename in os.listdir(test_folder) if filename.endswith(".mp4")])
pbar = tqdm(total=pbar_total, desc="Converting frames")

for filename in os.listdir(test_folder):
    if filename.endswith(".mp4"):
        category = extract_category(filename)
        category_output_folder = os.path.join(output_folder, category)
        os.makedirs(category_output_folder, exist_ok=True)
        video_path = os.path.join(test_folder, filename)
        convert_video_to_frames(video_path, category_output_folder, category, hc, pbar)

# Hand segment extraction for testing dataset

In [ ]:
os.makedirs(test_folder+'\\hand_segment_test', exist_ok=True)

if __name__ == '__main__':
    # Updating the path to your colored image
    image_path = test_folder+'\\frames_output'
    output_base_path = test_folder+'\\hand_segment_test'

    # Iterating over gesture folders
    for category_folder in os.listdir(image_path):
        for gesture_folder in os.listdir(os.path.join(image_path, category_folder)):
            gesture_folder_path = os.path.join(image_path, category_folder, gesture_folder)
            if os.path.isdir(gesture_folder_path):  # Checking if it's a directory
                # Creating the output folder for the current gesture
                output_gesture_folder = os.path.join(output_base_path, category_folder, gesture_folder)
                os.makedirs(output_gesture_folder, exist_ok=True)

                # Getting list of image files
                image_files = os.listdir(gesture_folder_path)
                # Setting up progress bar
                progress_bar = tqdm(total=len(image_files), desc=f"Processing {gesture_folder}")

                # Iterating over files in the current gesture folder
                for filename in image_files:
                    image_file_path = os.path.join(gesture_folder_path, filename)
                    # Reading the image
                    frame = cv2.imread(image_file_path)

                    # Performing hand segmentation
                    segmented_image = handsegment(frame)

                    # Determining the output file path
                    output_file_path = os.path.join(output_gesture_folder, filename)

                    # Converting frame to grayscale
                    gray_image = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)

                    # Saving the segmented image
                    cv2.imwrite(output_file_path, gray_image)

                    # Updating progress bar
                    progress_bar.update(1)

                # Closing progress bar
                progress_bar.close()